## TR question 2

Note: This requires running tr_q1 first

The order of the notebook is as follow, 
- Split train val test class
- Define a baseline
- Train model

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, f1_score
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from typing import Dict
import pandas as pd
import os
from transformers import Trainer, TrainingArguments

from datasets import Dataset

seed = 42
MAX_LENGTH = 512
NUM_DATAPOINT = 1000

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


I take the headtext and firsts characters of each paragraph and concatenatem together, such that each row in the dataset are representend by `MAX_LENGTH` characters

### Create a dataset

In [2]:
file_path = 'TRDataChallenge2023.zip'
extract_file_path = 'TRDataChallenge2023'
df = pd.read_json(os.path.join("/kaggle/input/test-t", f"{extract_file_path}.txt"), lines=True)
df = df.sample(NUM_DATAPOINT, random_state=42).reset_index(drop=True)

In [3]:
df

,documentId,postures,sections
0,I03d70fc0ce3311e8ae6bb4b0ae8dca5a,[Motion to Dismiss for Lack of Subject Matter ...,"[{'headtext': '', 'paragraphs': ['RANDOLPH, Se..."
1,I07f7893067c611ea9354eec9e02fecda,[Appellate Review],"[{'headtext': 'MEMORANDUM AND ORDER', 'paragra..."
2,Icf2416a2abce11e6b92bf4314c15140f,"[Motion to Renew, On Appeal]","[{'headtext': '', 'paragraphs': ['Appeal from ..."
3,Ic4f83620286111e8b25db53553f40f1b,[Review of Administrative Decision],"[{'headtext': 'MEMORANDUM AND JUDGMENT', 'para..."
4,Ia1075bf0779411e998e8870e22e55653,"[Appellate Review, Post-Trial Hearing Motion, ...","[{'headtext': 'OPINION', 'paragraphs': ['A pro..."
...,...,...,...
995,Ia0e3012a16b511e3981fa20c4f198a69,[On Appeal],"[{'headtext': '', 'paragraphs': ['Appeal by pl..."
996,I5bfac0801aee11ea942eedc092039568,"[Motion to Dismiss, On Appeal]","[{'headtext': 'OPINION', 'paragraphs': ['¶ 1 T..."
997,Ia21cd9c0fa4411e8a1b0e6625e646f8f,"[Motion to Dismiss, Motion to Dismiss for Lack...","[{'headtext': '', 'paragraphs': ['Jack Henry &..."
998,I74959190c9ba11e99c7da5bca11408d2,[Review of Administrative Decision],"[{'headtext': '', 'paragraphs': ['Florida Birt..."


Normally we `fit_transform` in the train set and `transform` on the test set. However here I `fit_transform` in the whole dataset to cover all of the labels, because some of them only have one instance (See first notebook)

In [4]:
mlb = MultiLabelBinarizer()
labels = pd.Series(np.array(mlb.fit_transform(df["postures"].values), dtype="float").tolist(), name="label_ids")

In [5]:
df['label_ids'] = labels

In [6]:
def clean_up_strings(max_len, sections):
    """
    Remove the \\u and clean up texts
    """
    cleaned_sections = []
    chars_per_section = max_len // len(sections)
    for section in sections:            
        cleaned_text = ""
        headtext = [section['headtext'].encode("ascii", "ignore").decode().strip()]
        cleaned_paragraph = [paragraph.encode("ascii", "ignore").decode().strip() for paragraph in section['paragraphs']]
        cleaned_text += ". ".join(headtext + cleaned_paragraph)        
        
        if (len(cleaned_text) < chars_per_section):
            cleaned_sections.append(cleaned_text[:len(cleaned_text)])
        else:
            last_space_index = cleaned_text[:chars_per_section].rfind(' ')
            cleaned_sections.append(cleaned_text[:last_space_index])  # last element that is a space

    cleaned_sections = '. '.join(cleaned_sections)

    return cleaned_sections

In [7]:
def test_clean_up_strings():
    # Test the function with a basic scenario
    max_len = 50
    sections = [
        {
            'headtext': "Sample Headline",
            'paragraphs': ["This is the first paragraph."]
        },
        {
            'headtext': "Sample Headline",
            'paragraphs': ["Second paragraph."]
        }
    ]
    cleaned_sections = clean_up_strings(max_len, sections)    
    expected_result = 'Sample Headline. This is. Sample Headline. Second'    
    assert cleaned_sections == expected_result

test_clean_up_strings()

In [8]:
df.isna().sum()

documentId    0
postures      0
sections      0
label_ids     0
dtype: int64

In [9]:
df["cleaned_text"] = df['sections'].map(lambda x: clean_up_strings(MAX_LENGTH, x))

In [10]:
df

,documentId,postures,sections,label_ids,cleaned_text
0,I03d70fc0ce3311e8ae6bb4b0ae8dca5a,[Motion to Dismiss for Lack of Subject Matter ...,"[{'headtext': '', 'paragraphs': ['RANDOLPH, Se...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",". RANDOLPH, Senior Circuit Judge: Frank Palaci..."
1,I07f7893067c611ea9354eec9e02fecda,[Appellate Review],"[{'headtext': 'MEMORANDUM AND ORDER', 'paragra...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","MEMORANDUM AND ORDER. In 2013, defendant plead..."
2,Icf2416a2abce11e6b92bf4314c15140f,"[Motion to Renew, On Appeal]","[{'headtext': '', 'paragraphs': ['Appeal from ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",". Appeal from an order of the Supreme Court, K..."
3,Ic4f83620286111e8b25db53553f40f1b,[Review of Administrative Decision],"[{'headtext': 'MEMORANDUM AND JUDGMENT', 'para...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",MEMORANDUM AND JUDGMENT. Proceeding pursuant t...
4,Ia1075bf0779411e998e8870e22e55653,"[Appellate Review, Post-Trial Hearing Motion, ...","[{'headtext': 'OPINION', 'paragraphs': ['A pro...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",OPINION. A prosecutor. I.BACKGROUND. After a. ...
...,...,...,...,...,...
995,Ia0e3012a16b511e3981fa20c4f198a69,[On Appeal],"[{'headtext': '', 'paragraphs': ['Appeal by pl...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",. Appeal by plaintiffs and nonparty Amos Weinb...
996,I5bfac0801aee11ea942eedc092039568,"[Motion to Dismiss, On Appeal]","[{'headtext': 'OPINION', 'paragraphs': ['¶ 1 T...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","OPINION. 1 The claimant, Paul D. Heck, appeals..."
997,Ia21cd9c0fa4411e8a1b0e6625e646f8f,"[Motion to Dismiss, Motion to Dismiss for Lack...","[{'headtext': '', 'paragraphs': ['Jack Henry &...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",". Jack Henry & Associates, Inc. (Jack Henry) a..."
998,I74959190c9ba11e99c7da5bca11408d2,[Review of Administrative Decision],"[{'headtext': '', 'paragraphs': ['Florida Birt...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",. Florida Birth-Related Neurological Injury Co...


### Create a dataset

In [11]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForSequenceClassification.from_pretrained('poltextlab/xlm-roberta-large-english-legal-cap',
                                                           num_labels= len(mlb.classes_),
                                                           problem_type="multi_label_classification",
                                                           ignore_mismatched_sizes=True)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at poltextlab/xlm-roberta-large-english-legal-cap and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([22, 1024]) in the checkpoint and torch.Size([90, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([22]) in the checkpoint and torch.Size([90]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def tokenize(examples):
    encoding = tokenizer(examples["cleaned_text"], padding="max_length", truncation=True, max_length=512)    
    
    return encoding

In [13]:

train_dataset = Dataset.from_pandas(df)
tokenized_datasets = train_dataset.map(tokenize)
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2, seed=seed)


  0%|          | 0/1000 [00:00<?, ?ex/s]

### Train

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


In [15]:

training_args = TrainingArguments(
    per_device_train_batch_size=3,
    output_dir='./output', 
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    num_train_epochs=10,
    logging_dir='./logs',
    seed=seed
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.079900,0.055265
1000,0.051900,0.055087
1500,0.050700,0.055243
2000,0.050600,0.055702
2500,0.051600,0.055889


TrainOutput(global_step=2670, training_loss=0.05653157680668634, metrics={'train_runtime': 2022.5813, 'train_samples_per_second': 3.955, 'train_steps_per_second': 1.32, 'total_flos': 7457667661824000.0, 'train_loss': 0.05653157680668634, 'epoch': 10.0})

### Evaluation

#### From model

In [16]:
results = trainer.predict(tokenized_datasets["test"])

In [17]:
values = np.argmax(results.predictions, axis=1)
n_values = len(mlb.classes_)
prediction = np.eye(n_values)[values]

In [18]:
f1_score(y_true=tokenized_datasets["test"]['label_ids'], y_pred=prediction, average='weighted')

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.17730245902516834

#### From a Baseline
We noticed from 1st question
- Most common class: Appellate Review
- Most common number of labels: 1

Therefore, the baseline would be to predict everything with "Appellate Review"

In [19]:
baseline_pred = mlb.transform([['Appellate Review']] * len(tokenized_datasets["test"]))

In [20]:
multilabel_confusion_matrix(y_true=tokenized_datasets["test"]['label_ids'], y_pred=baseline_pred)

array([[[  0, 127],
        [  0,  73]],

       [[200,   0],
        [  0,   0]],

       [[200,   0],
        [  0,   0]],

       [[199,   0],
        [  1,   0]],

       [[199,   0],
        [  1,   0]],

       [[198,   0],
        [  2,   0]],

       [[199,   0],
        [  1,   0]],

       [[200,   0],
        [  0,   0]],

       [[200,   0],
        [  0,   0]],

       [[195,   0],
        [  5,   0]],

       [[199,   0],
        [  1,   0]],

       [[199,   0],
        [  1,   0]],

       [[198,   0],
        [  2,   0]],

       [[199,   0],
        [  1,   0]],

       [[200,   0],
        [  0,   0]],

       [[195,   0],
        [  5,   0]],

       [[200,   0],
        [  0,   0]],

       [[198,   0],
        [  2,   0]],

       [[198,   0],
        [  2,   0]],

       [[200,   0],
        [  0,   0]],

       [[198,   0],
        [  2,   0]],

       [[199,   0],
        [  1,   0]],

       [[200,   0],
        [  0,   0]],

       [[200,   0],
        [  0, 

In [21]:
f1_score(y_true=tokenized_datasets["test"]['label_ids'], y_pred=baseline_pred, average='weighted')

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.1216208505928132